<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark/challenges/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 1
##  Implement INGESTION process
- Set up path in the "lake"
  - !mkdir -p /content/lake/bronze

- Read data from API https://api.carrismetropolitana.pt/
  - Endpoints:
    - vehicles
    - lines
    - municipalities
  - Use StructFields to enforce schema

- Transformations
  - vehicles
    - create "date" extracted from "timestamp" column (format: hh24miss)

- Write data as PARQUET into the BRONZE layer (/content/lake/bronze)
  - Partition "vehicles" by "date" column
  - Paths:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# Setting up PySpark

In [53]:
%pip install pyspark

In [54]:
# Import SparkSession

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Carla_Ferreira_API_challenge').config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

In [55]:
import requests
from pyspark.sql.types import *

def readFromAPI(url: str, schema: StructType = None):
  response = requests.get(url)
  rdd = sc.parallelize(response.json())

  if schema:
    df = spark.read.schema(schema).json(rdd)
  else:
    df = spark.read.json(rdd)
  return df

In [57]:
#Schemas
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)
                             ])


lines_schema = StructType([StructField('_corrupt_record', StringType(), True),
                           StructField('color', StringType(), True),
                           StructField('facilities', StringType(), True),
                           StructField('id', StringType(), True),
                           StructField('localities', ArrayType(StringType()), True),
                           StructField('long_name', StringType(), True),
                           StructField('municipalities', ArrayType(StringType()), True),
                           StructField('patterns', ArrayType(StringType()), True),
                           StructField('routes', StringType(), True),
                           StructField('short_name', StringType(), True),
                           StructField('text_color', StringType(), True)
                           ])

municipalities_schema = StructType([StructField('district_id', StringType(), True),
                                    StructField('district_name', StringType(), True),
                                    StructField('id', StringType(), True),
                                    StructField('name', StringType(), True),
                                    StructField('prefix', StringType(), True),
                                    StructField('region_id', StringType(), True),
                                    StructField('region_name', StringType(), True)
                                    ])

In [58]:
vehicles = readFromAPI("https://api.carrismetropolitana.pt/vehicles", schema = vehicle_schema)
lines = readFromAPI("https://api.carrismetropolitana.pt/lines", schema = lines_schema)
municipalities = readFromAPI("https://api.carrismetropolitana.pt/municipalities", schema = municipalities_schema)

In [59]:
# create "date" extracted from "timestamp" column (format: hh24miss) -- this format doesnt make sense, the correct is yyyyMMdd
from pyspark.sql.functions import date_format
vehicles = vehicles.withColumn("date", date_format('timestamp', "yyyyMMdd"))

vehicles.show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+--------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|    date|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+--------+
|    145|20241121-64010054...| IN_TRANSIT_TO|44|12535|38.700203|   4504|-9.005495|  4504_0_1|  4504_0|            SCHEDULED|121890234560|      0.0| 100138|2024-11-21 21:28:20|4504_0_1|2100|213...|20241121|
|      7|           1_1033-11| IN_TRANSIT_TO| 41|1176|38.763058|   1710|-9.197116|  1710_3_2|  1710_3|            SCHEDULED|        1140|0.2777778| 110045|2024-11-21 21:28:06|1

In [60]:
from pyspark.sql import functions as F

vehicles.select("date").distinct().agg(F.min("date").alias("min_date")).show()
vehicles.select("date").distinct().agg(F.max("date").alias("max_date")).show()

# there's only one day in this file.

+--------+
|min_date|
+--------+
|20241121|
+--------+

+--------+
|max_date|
+--------+
|20241121|
+--------+



Write data as PARQUET into the BRONZE layer (/content/lake/bronze)

*   Partition "vehicles" by "date" column
*   Paths: /content/lake/bronze/vehicles, /content/lake/bronze/lines
, /content/lake/bronze/municipalities

Make sure there is only 1 single parquet created /
Use overwrite as write mode

Declare the directory to write the data that came from API carrismetropolitana

In [61]:
!mkdir -p /content/lake/bronze

path_vehicles = "/content/lake/bronze/vehicles"
path_lines = "/content/lake/bronze/lines"
path_municipalities = "/content/lake/bronze/municipalities"

In [62]:
(vehicles
.coalesce(1)
.write
.mode("overwrite")
.partitionBy("date")
.format("parquet")
.save(path_vehicles)
)


(lines
.write
.mode("overwrite")
.format("parquet")
.save(path_lines)
)


(municipalities
.write
.mode("overwrite")
.format("parquet")
.save(path_municipalities)
)

# CHALLENGE 2
##  Implement CLEANSING process
- Set up path in the "lake"
  - !mkdir -p /content/lake/silver

- Read data from BRONZE layer as PARQUET:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities

- Transformations
  - vehicles
    - rename "lat" and "lon" to "latitude" and "longitude" respectively
    - remove possible duplicates
    - remove rows when the column CURRENT_STATUS is null
    - remove any corrupted record
  - lines
    - remove duplicates
    - remove rows when the column LONG_NAME is null
    - remove any corrupted record
  - municipalities
    - remove duplicates
    - remove rows when the columns NAME or DISTRICT_NAME are null
    - remove any corrupted record

- Write data as PARQUET into the SILVER layer (/content/lake/silver)
  - Partition "vehicles" by "date"(created in the ingestion)
  - Paths:
    - vehicles - path: /content/lake/silver/vehicles
    - lines - path: /content/lake/silver/lines
    - municipalities - path: /content/lake/silver/municipalities

In [63]:
vehicles = spark.read.parquet("/content/lake/bronze/vehicles", schema = vehicle_schema)

In [64]:
lines = spark.read.parquet("/content/lake/bronze/lines", schema = lines_schema)

In [65]:
municipalities = spark.read.parquet("/content/lake/bronze/municipalities", schema = municipalities_schema)

In [66]:
from pyspark.sql.functions import *

# Vehicles
# rename "lat" and "lon" to "latitude" and "longitude" respectively
vehicles = vehicles.withColumnRenamed("lat", "latitude").withColumnRenamed("lon", "longitude")

# remove possible duplicates
vehicles = vehicles.drop_duplicates()
# remove rows when the column CURRENT_STATUS is null
vehicles = vehicles.filter(vehicles["current_status"].isNotNull())
# remove any corrupted record (????)

In [67]:
from pyspark.sql.functions import *

# Lines
# remove duplicates
lines = lines.drop_duplicates()
# remove rows when the column LONG_NAME is null
lines = lines.filter(lines["long_name"].isNotNull())
# remove any corrupted record
lines = lines.filter(lines["_corrupt_record"].isNull())

In [68]:
# Municipalities
# remove duplicates
municipalities = municipalities.drop_duplicates()
# remove rows when the columns NAME or DISTRICT_NAME are null
municipalities = municipalities.filter(municipalities["name"].isNotNull() & municipalities["district_name"].isNotNull())
# remove any corrupted record

In [69]:
!mkdir -p /content/lake/silver

In [70]:
path_vehicles = "/content/lake/silver/vehicles"
path_lines = "/content/lake/silver/lines"
path_municipalities = "/content/lake/silver/municipalities"

In [71]:
(vehicles
.coalesce(1)
.write
.mode("overwrite")
.partitionBy("date")
.format("parquet")
.save(path_vehicles)
)


(lines
.write
.mode("overwrite")
.format("parquet")
.save(path_lines)
)


(municipalities
.write
.mode("overwrite")
.format("parquet")
.save(path_municipalities)
)

# CHALLENGE 3
##  Implement ENRICH process
- Set up path in the "lake"
  - !mkdir -p /content/lake/gold

- Read data from SILVER layer
  - Paths:
    - vehicles - path: /content/lake/silver/vehicles
    - lines - path: /content/lake/silver/lines
    - municipalities - path: /content/lake/silver/municipalities
  - Use StructFields to enforce schema

- Enrichment
  - Enrich vehicles dataset with information from the line and municipalities
    - join vehicles with lines and municipalities
      - select all columns from vehicles + lines.long_name (name: line_name, format:string) + municipalities.name (name: municipality_name, format: array)
      - Note that "municipalities.name" is an array

- Write data as PARQUET into the GOLD layer (/content/lake/gold)
  - Dataset name: vehicles_enriched
  - Partition "vehicles_enriched" by "date" column
  - Paths:
    - vehicles - path: /content/lake/gold/vehicles_enriched
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

In [72]:
vehicles = spark.read.parquet("/content/lake/silver/vehicles", schema = vehicle_schema)

In [73]:
lines = spark.read.parquet("/content/lake/silver/lines", schema = lines_schema)

In [74]:
municipalities = spark.read.parquet("/content/lake/silver/municipalities", schema = municipalities_schema)

In [76]:
# Extract info from the array in municiples
from pyspark.sql.functions import explode
lines_treated = lines.select(
    "long_name",
    explode(lines.municipalities).alias("municipality")  # Explode 'municipalities' into individual rows
)

In [77]:
lines_treated.show()

+--------------------+------------+
|           long_name|municipality|
+--------------------+------------+
|Casal de Cambra (...|        1115|
|Casal de Cambra (...|        1111|
|Ericeira (Termina...|        1109|
|Mitrena (Portucel...|        1512|
|Lisboa (Sete Rios...|        1512|
|Lisboa (Sete Rios...|        1511|
|Lisboa (Sete Rios...|        1504|
|Lisboa (Sete Rios...|        1510|
|Lisboa (Sete Rios...|        1503|
|Lisboa (Sete Rios...|        1106|
|Amadora (Hospital...|        1115|
|Amadora (Hospital...|        1111|
|  Ervideira - Gradil|        1109|
|Algeruz - Setúbal...|        1508|
|Algeruz - Setúbal...|        1512|
|Monte Abraão (Est...|        1111|
|Casal Cambra - Va...|        1111|
|Casal Cambra - Va...|        1116|
|Sto. Ant. Cavalei...|        1107|
|Lisboa (C. Grande...|        1106|
+--------------------+------------+
only showing top 20 rows



In [ ]:
vehicles_municipalities = vehicles.join(lines, vehicles['line_id'] = lines['id'], how = 'left')